# Week 9 - Root finding

Problem: to find a *root* of a some function $F : \mathbb{R} \rightarrow \mathbb{R}$. i.e. to find some $x_*$ such that

$$F(x_*) = 0.$$

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np


def F(x):
    return x * np.sin(x) - 0.5 * x


x_plot = np.linspace(-np.pi, np.pi, 1000)
fig, ax = plt.subplots(1)
ax.plot(x_plot, F(x_plot), "k-")
ax.axhline(0, color="#888888")
ax.axvline(0, color="#888888")
ax.set_xlim(x_plot[0], x_plot[-1])
ax.set_xlabel("$x$", fontsize="x-large")
ax.set_ylabel("$F(x)$", fontsize="x-large")

## Bisection

If

- We can choose $a$ and $b$ such that $F(a) F(b) \le 0$
- $F$ is continuous on $[a, b]$

then we must have at least one root somewhere in $[a, b]$.

General idea:

1. Assume we have some appropriate $a$, $b$
2. Let $c$ be the mid-point between them, $c \gets (a + b)/2$
3. 'Zoom in' onto one side, either $b \gets c$ or $a \gets c$, so that we keep $F(a) F(b) \le 0$
4. Iterate until some *convergence criteria* are satisfied

In [ ]:
a, b = 2, 3

In [ ]:
zoom = True

if F(a) * F(b) > 0:
    raise ValueError("Invalid (a, b)")
c = (a + b) / 2

if zoom:
    x_plot = np.linspace(a - 0.1 * (b - a), b + 0.1 * (b - a), 1000)
else:
    x_plot = np.linspace(-np.pi, np.pi, 1000)
fig, ax = plt.subplots(1)
ax.plot(x_plot, F(x_plot), "k-")
ax.axhline(0, color="#888888")
ax.axvline(0, color="#888888")
ax.axvline(a, color="r", linestyle="-")
ax.axvline(b, color="r", linestyle="-")
ax.axvline(c, color="r", linestyle=":")
ax.set_xlim(x_plot[0], x_plot[-1])
ax.set_xlabel("$x$", fontsize="x-large")
ax.set_ylabel("$F(x)$", fontsize="x-large")

if F(a) * F(c) <= 0:
    print("Chose the left side")
    b = c
else:
    print("Chose the right side")
    a = c

In [ ]:
from itertools import count


def bisection(F, a, b, *, tol=1e-10):
    for it in count(start=1):
        if F(a) * F(b) > 0:
            raise ValueError("Invalid (a, b)")
        c = (a + b) / 2
        # print(f"{it=:4d} {abs(b - a)=:6e} {abs(F(c))=:.6e}")
        if abs(b - a) < tol:
            return c
        if F(a) * F(c) <= 0:
            b = c
        else:
            a = c


print(f"{bisection(F, -0.1, 0.1)=}")
print(f"{bisection(F, 0.1, 2)=}")
print(f"{bisection(F, 2, 3)=}")

## Regula falsi

Regula falsi (or the 'false position method') is very similar to bisection, but instead chooses $c$ by linear interpolation. For $F(a) F(b) < 0$ we let

$$p(x) = \alpha x + \beta,$$

with

$$p(a) = \alpha a + \beta = F(a),$$
$$p(b) = \alpha b + \beta = F(b).$$

Defining $c$ such that $p(c) = 0$ we find

$$c = \frac{a F(b) - b F(a)}{F(b) - F(a)},$$

noting that we never divide by zero since $F(a) F(b) < 0$, and so $F(a)$ and $F(b)$ are non-zero with differing signs.

## Fixed-point iteration

Fixed-point iteration, when used for root finding, is a broad family of methods based on the idea of choosing some *iteration function* $G(x)$, some *initial guess* $x_0$, and then iterating so that

$$x_{n + 1} = G(x_n) \qquad \text{for} ~ n \ge 0.$$

$G$ is defined so that if we are at a *fixed-point* $x_*$ of $G$, i.e. if we have

$$x_* = G(x_*),$$

then we have found a root of $F$

$$F(x_*) = 0.$$

For fixed-point iteration to converge to a fixed-point $x_*$, it is sufficient for there to be some neighbourhood $[a, b]$ with $b > a$ such that

- $x_* \in (a, b)$
- $G$ is continuous on $[a, b]$
- $G$ is differentiable on $(a, b)$
- $|G'(x)| \le K$ for all $x$ in $(a, b)$ for some constant $K$ with $0 < K < 1$
- $x_n$ is sufficiently close to $x_*$ for some $n$ (e.g. $n = 0$)

Simple way to construct $G$: re-arrange, e.g.

$$F(x_*) = x_* \sin x_* - \frac{1}{2} x_* = 0$$

can be re-arranged to

$$x_* = 2 x_* \sin x_*,$$

suggesting the fixed-point iteration

$$x_{n + 1} = 2 x_n \sin x_n.$$

In [ ]:
eps = 1e-10
max_it = 50
x = 3
for it in range(1, max_it + 1):
    x = 2 * x * np.sin(x)
    print(f"{it=:4d} {x=:6e} {abs(F(x))=:.6e}")
    if abs(F(x)) < eps:
        break

Another re-arrangement suggests

$$x_{n + 1} = \frac{x_n}{2 \sin x_n}.$$

In [ ]:
eps = 1e-10
max_it = 50
x = 3
for it in range(1, max_it + 1):
    x = x / (2 * np.sin(x))
    print(f"{it=:4d} {x=:6e} {abs(F(x))=:.6e}")
    if abs(F(x)) < eps:
        break

and a third suggests

$$x_{n + 1} = \frac{2 x_n^2 \cos x_n}{2 \sin x_n + 2 x_n \cos x_n - 1}.$$

In [ ]:
eps = 1e-10
max_it = 50
x = 3
for it in range(1, max_it + 1):
    x = 2 * x ** 2 * np.cos(x) / (2 * np.sin(x) + 2 * x * np.cos(x) - 1)
    print(f"{it=:4d} {x=:6e} {abs(F(x))=:.6e}")
    if abs(F(x)) < eps:
        break

Question: how should we choose the iteration function?